In [ ]:
#Importing all the libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset,DataLoader
from torchvision import models,datasets,transforms

from tqdm import tqdm
import os
from PIL import Image
import matplotlib.pyplot as plt
import math
import random
import pickle

In [ ]:
#Checking if a GPU with CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
#Creating a custom dataset class that generates the noisy counterpart of the clean image and returns both of them
class CustomDataset(Dataset):
  def __init__(self,dir_name,train_flag,sigma):
    """
    train_flag is True for the Train dataset and False for the evaluation dataset
    sigma is the noise level
    """
    super().__init__()
    #Defining the transforms for the train and test datasets
    self.test_transform = transforms.Compose([transforms.ToTensor()])

    self.train_flag = train_flag

    #Downloading the train and test datasets
    if train_flag:
      pfile = open(dir_name, 'rb')     #opening the pickle file
      self.dataset = pickle.load(pfile) #Reading the tensors from the pickle file
      pfile.close() #Closing the pickle file

    else:
      self.dataset = []
      for cl in os.listdir(dir_name):
        for img_name in os.listdir(os.path.join(dir_name,cl)):
          self.dataset.append(os.path.join(dir_name,cl,img_name))

    self.sigma = sigma

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    if self.train_flag:
      clean_img = self.dataset[index].unsqueeze(0) #of shape (1,40,40)

    else:
      im = Image.open(self.dataset[index]).convert("L") #PIL object in grayscale
      clean_img = self.test_transform(im) #of shape (1,h,w)

    #Generating the noisy image
    noisy_img = (torch.randn(clean_img.shape)*(self.sigma/255.))+clean_img
    return clean_img, noisy_img



In [ ]:
#Defining a Block of the DnCNN
class Block(nn.Module):
  def __init__(self,k=3,p=1,c=64):
    super().__init__()
    self.conv = nn.Conv2d(in_channels=c,out_channels=c,kernel_size=k,padding=p,bias=False) #same padding convolution
    self.norm = nn.BatchNorm2d(c) #batch normalization
    self.relu = nn.ReLU(inplace=True) #activation function

  def forward(self,x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.relu(x)
    return x

In [ ]:
#Defining the DnCNN model
class DCNN(nn.Module):
  def __init__(self,k=3,p=1,c=64,l=17,in_c=1):
    super().__init__()
    layers = [nn.Conv2d(in_channels=in_c,out_channels=c,kernel_size=k,padding=p,bias=False), #First same padding convolution layer
              nn.ReLU(inplace=True)]
    layers.extend([Block(k,p,c) for _ in range(l-2)]) #Adding all the "Blocks" to the model
    layers.append(nn.Conv2d(in_channels=c,out_channels=in_c,kernel_size=k,padding=p,bias=False)) #Last same padding convolution layer

    self.all = nn.Sequential(*layers)

  def forward(self,x):
    out = self.all(x)
    return x-out


In [ ]:
#Function for supervised training of the model
def train(criterion,optimizer,model,device,train_loader):
  """
  criterion is the loss function
  optimizer is the optimization algorithm used
  model is the denoiser model
  device is either CPU or GPU(cuda)
  train_loader is the DataLoader containing the training dataset
  """

  model.train()
  loop = tqdm(train_loader)  #Used to visualized the progress in training
  cur_loss = 0.0

  for i,(clean,noisy) in enumerate(loop):   #iterating batch-by-batch through the dataset
    clean,noisy = clean.to(device), noisy.to(device) #Moving over the data to the "device"
    clean_pred = model(noisy) #Passing the data through the model
    loss = criterion(clean_pred,clean) #Computing the loss

    optimizer.zero_grad() #Zeroing all the previous gradients
    loss.backward() #Computing the gradients for the current iteration
    optimizer.step() #Updating the weights of the model

    cur_loss += loss.item() #Keeping track of the loss
    loop.set_postfix(loss=cur_loss/(i+1)) #Printing the cumulative loss after each iteration

In [ ]:
#Function for testing the model
def test(criterion,model,device,test_loader):
  """
  criterion is the function used to compute PSNR
  model is the denoiser model
  device is either CPU or GPU(cuda)
  test_loader is the DataLoader containing the test dataset
  """

  model.eval()
  loop = tqdm(test_loader) #Used to visualized the progress in testing
  total_mse = []

  with torch.no_grad(): #Ensures that the gradients are not computed
    for i,(clean,noisy) in enumerate(loop): #iterating batch-by-batch through the dataset
      clean,noisy = clean.to(device).float(), noisy.to(device).float() #Moving over the data to the "device"
      out = model(noisy) #Passing the data through the model

      clean_pred = torch.clamp(out,min=0.0,max=1.0) #Clips all the values greater than 1 or less than 0
      loss = (criterion(clean_pred,clean).mean(axis=(1,2,3))).tolist() #Computing MSE at an image level
      total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean() #Computing the PSNR using the corresponding MSE values

  print(f"The PSNR is {psnr}")
  return psnr.item()

In [ ]:
#Function to computer the number of parameters in a model
def number_of_parameters(model):
    return sum(params.numel() for params in model.parameters() if params.requires_grad)

In [ ]:
#Wrapper function to train and evaluate the denoiser model
def wrapper(sigma):
  print(f"This is for sigma of {sigma}")

  #Defines the loaders for the train and test set
  train_set = CustomDataset("/content/drive/MyDrive/R2R_ML/Covid_dataset/Covid_train.pkl",True,sigma)
  test_set = CustomDataset("/content/drive/MyDrive/R2R_ML/Covid_dataset/split4",False,sigma)

  train_loader = DataLoader(train_set,batch_size=128,shuffle=True,num_workers=64)
  test_loader = DataLoader(test_set,batch_size=1,shuffle=False,num_workers=1)

  print(f"The number of images in the train set is {len(train_set)}")
  print(f"The number of images in the test set is {(len(test_set))}")

  #Defining the model, loss function and optimizer
  model = DCNN().to(device)
  criterion_train = nn.MSELoss()
  criterion_test = nn.MSELoss(reduce=False)
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  epochs = 10

  print(f"The model has {number_of_parameters(model)} parameters")
  #Computing the PSNR between the noisy and clean image
  total_mse = []
  with torch.no_grad():
      for i,(clean,noisy) in enumerate(test_loader):
        clean,noisy = clean.to(device), noisy.to(device)
        loss = (criterion_test(noisy,clean).mean(axis=(1,2,3))).tolist()
        total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean()
  print(f"The PSNR for an untrained densoiser is {psnr}")

  #Iterating through the epochs
  best_psnr = 0.0
  for epoch in range(epochs):
    print(f"The current epoch is {epoch}")
    train(criterion_train,optimizer,model,device,train_loader)
    cur_psnr = test(criterion_test,model,device,test_loader)
    if cur_psnr>best_psnr: #Saving the model with the best PSNR value
      best_psnr = cur_psnr
      torch.save(model.state_dict(), "Supervised"+str(epoch)+"_"+str(round(cur_psnr,2))+"_"+ str(sigma) + ".pt")


In [ ]:
#For sigma value of 10
wrapper(10)

This is for sigma of 10


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The number of images in the train set is 129664
The number of images in the test set is 517
The model has 556032 parameters


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


The PSNR for an untrained densoiser is 28.131553649902344
The current epoch is 0


...

The PSNR is 39.56069564819336
The current epoch is 8


100%|██████████| 517/517 [00:13<00:00, 37.02it/s]


The PSNR is 38.736732482910156
The current epoch is 9


100%|██████████| 517/517 [00:13<00:00, 37.19it/s]


The PSNR is 39.44612121582031


In [ ]:
#For sigma value of 25
wrapper(25)

This is for sigma of 25
The number of images in the train set is 129664
The number of images in the test set is 517
The model has 556032 parameters
The PSNR for an untrained densoiser is 20.172510147094727
The current epoch is 0


100%|██████████| 517/517 [00:14<00:00, 36.52it/s]


The PSNR is 34.43827438354492
The current epoch is 1


100%|██████████| 517/517 [00:14<00:00, 35.39it/s]


The PSNR is 33.95232391357422
The current epoch is 2


...

The PSNR is 21.463497161865234
The current epoch is 8


100%|██████████| 517/517 [00:14<00:00, 36.23it/s]


The PSNR is 34.08458709716797
The current epoch is 9


100%|██████████| 517/517 [00:14<00:00, 36.53it/s]


The PSNR is 35.09721374511719


In [ ]:
#For sigma value of 50
wrapper(50)

This is for sigma of 50
The number of images in the train set is 129664
The number of images in the test set is 517
The model has 556032 parameters
The PSNR for an untrained densoiser is 14.150960922241211
The current epoch is 0


100%|██████████| 517/517 [00:14<00:00, 36.00it/s]


The PSNR is 30.94282341003418
The current epoch is 1


100%|██████████| 517/517 [00:13<00:00, 37.76it/s]


The PSNR is 31.24430274963379
The current epoch is 2


...

The PSNR is 32.79485321044922
The current epoch is 8


100%|██████████| 517/517 [00:14<00:00, 35.98it/s]


The PSNR is 33.19359588623047
The current epoch is 9


100%|██████████| 517/517 [00:14<00:00, 36.31it/s]


The PSNR is 33.150550842285156
